## Pipeline for downloading any number of last available pairs of audio and chat of streams

In [ ]:
streamer_name = "rostislav_999"  # Replace with the streamer's name
num_streams_to_download = 10     # Number of most recent streams to download
client_id = "your-client-id"     # Your Twitch API client ID
quality = "720p60"               # Preferred video quality

### Install necessary libraries

In [ ]:
!apt-get update && sudo apt-get install -y ffmpeg  # Install ffmpeg
!pip3 install tdh-tcd  # Install the tdh-tcd library for downloading Twitch chat

In [ ]:
import os
if not os.path.exists("twitch-dl.2.11.0.pyz"):
    print("Downloading twitch-dl...")
    !wget https://github.com/ihabunek/twitch-dl/releases/download/2.11.0/twitch-dl.2.11.0.pyz

### Get list of recent VODs for the streamer

In [ ]:
!python3 twitch-dl.2.11.0.pyz videos {streamer_name} --limit {num_streams_to_download} > vods.txt

In [ ]:
with open("vods.txt", "r") as f:
    lines = f.readlines()

vod_ids = []
for line in lines:
    if line.startswith("https://www.twitch.tv/videos/"):
        vod_id = line.strip().split("/")[-1]
        vod_ids.append(vod_id)

print(f"Found {len(vod_ids)} VODs to download")

### Process each VOD

In [ ]:
for vod_id in vod_ids:
    print(f"\nProcessing VOD {vod_id}")

    # Download the Twitch video
    print("Downloading video...")
    !python3 twitch-dl.2.11.0.pyz download https://www.twitch.tv/videos/{vod_id} --output "{vod_id}.mkv" --quality "{quality}"

    # Extract audio if needed
    # print("Extracting audio...")
    # !ffmpeg -i {vod_id}.mkv -vn -acodec pcm_s16le {vod_id}.wav

    # Download chat
    print("Downloading chat...")
    !python3 -m tcd {vod_id} --client-id "{client_id}"

    # Convert to jpg if needed
    # nth_frame = 30
    # output_folder = f"{vod_id}_frames"
    # os.makedirs(output_folder, exist_ok=True)
    # os.system(f"ffmpeg -i {vod_id}.mkv -vf \"select='not(mod(n,{nth_frame}))'\" -vsync 0 {output_folder}/ frame_%05d.jpg")

    # Remove the .mkv file
    # print("Cleaning up...")
    # !rm {vod_id}.mkv

    # print(f"Finished processing VOD {vod_id}")

print("\nAll downloads completed!")